In [1]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Name and ID (Section 02) Text Analytics CISB5123 
# Devendran a/l Pathamanathan(SW01083554)
# Ishamie Syazwina binti Mohd Fauzi (IS01083940)

# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set the path to EdgeDriver (update this path)
edge_driver_path = r"C:\Users\Skylevender\Desktop\test\msedgedriver.exe"

# Initialize Edge WebDriver
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)
wait = WebDriverWait(driver, 10)

# Amazon product URL
product_url = "https://www.amazon.com/product-reviews/B0D86FVG5R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
driver.get(product_url)
time.sleep(5)  # Wait for the page to load

def extract_reviews():
    reviews = []
    
    try:
        review_blocks = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, '.review')))
        
        for block in review_blocks:
            try:
                reviewer_name = block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
                review_date = block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()
                review_content = block.find_element(By.CSS_SELECTOR, '.review-text-content span').text.strip()

                reviews.append({
                    'Reviewer Name': reviewer_name,
                    'Review Date': review_date,
                    'Review Content': review_content
                })
            except Exception as e:
                print("Error extracting a review:", e)
                continue  # Skip problematic reviews
                
    except Exception as e:
        print("Error loading reviews:", e)
    
    return reviews

# Initialize the list before using it
all_reviews = []

# Scrape reviews from multiple pages
for page in range(1, 6):  # Adjust the range as needed
    print(f"Scraping page {page}...")
    all_reviews.extend(extract_reviews())  # Now all_reviews is defined
    try:
        next_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//a[contains(text(), 'Next page')]")))  # Adjusted for new button
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(5)  # Wait for next page to load
    except Exception:
        print("No more pages or 'Next page' button not found.")
        break  # Stop if there's no "Next page" button

# Convert to DataFrame
df_reviews = pd.DataFrame(all_reviews)

# Save to CSV
df_reviews.to_csv('AMAZON_REVIEW.csv', index=False)

print("Reviews saved successfully to AMAZON_REVIEW.csv")

# Close the browser
driver.quit()


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Reviews saved successfully to AMAZON_REVIEW.csv
